# PulsarPrediction

## Introduction

In [19]:
suppressMessages(library(tidyverse))
suppressMessages(library(repr))
suppressMessages(library(tidymodels))
suppressMessages(library(qdap))
options(repr.matrix.max.rows = 6)

ERROR: Error in library(qdap): there is no package called ‘qdap’


In [103]:
pulsar_training = read_csv("pulsar_data_train.csv")
pulsar_training = na.omit(pulsar_training)


Parsed with column specification:
cols(
  `Mean of the integrated profile` = col_double(),
  `Standard deviation of the integrated profile` = col_double(),
  `Excess kurtosis of the integrated profile` = col_double(),
  `Skewness of the integrated profile` = col_double(),
  `Mean of the DM-SNR curve` = col_double(),
  `Standard deviation of the DM-SNR curve` = col_double(),
  `Excess kurtosis of the DM-SNR curve` = col_double(),
  `Skewness of the DM-SNR curve` = col_double(),
  target_class = col_double()
)



In [104]:
pulsar_names = c()
for(n in names(pulsar_training)){
    acroynm = ''
    for(word in str_split(n, ' ')){
        acroynm = paste(acroynm,substr(word,1,1), sep = '', collapse='')
    }
    pulsar_names = c(pulsar_names,acroynm)
}
names(pulsar_training)<-pulsar_names
pulsar_training




Motip,Sdotip,Ekotip,Sotip,MotDc,SdotDc,EkotDc,SotDc,t
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
121.15625,48.37297,0.3754847,-0.01316549,3.168896,18.39937,7.449874,65.15930,0
76.96875,36.17556,0.7128979,3.38871856,2.399666,17.57100,9.414652,102.72297,0
130.58594,53.22953,0.1334083,-0.29724164,2.743311,22.36255,8.508364,74.03132,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
116.0312,43.21385,0.66345569,0.4330880,0.7851171,11.62815,17.055215,312.20433,0
135.6641,49.93375,-0.08994031,-0.2267262,3.8595318,21.50150,7.398395,62.33402,0
120.7266,50.47226,0.34617808,0.1847972,0.7692308,11.79260,17.662222,329.54802,0


In [155]:
# aov(t ~ Motip, data=pulsar_training)
summary(aov(t ~ Motip, data=pulsar_training))
# residuals(aov(t ~ Motip, data=pulsar_training))
aov.models = pulsar_training %>%
  map(~ summary(aov(pulsar_training$t ~ .x))[[1]][["F value"]][1])

as_tibble(aov.models) %>%
pivot_longer(cols=Motip:t, names_to="predictor") %>%
filter(predictor != 't') %>%
arrange(desc(value)) %>%
head(4)

summary(aov(t ~ Ekotip + Sotip + Motip + SdotDc, data=pulsar_training))

              Df Sum Sq Mean Sq F value Pr(>F)    
Motip          1  352.6   352.6    7794 <2e-16 ***
Residuals   9271  419.4     0.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

predictor,value
<chr>,<dbl>
Ekotip,15482.646
Sotip,9148.079
Motip,7794.219
SdotDc,2979.422


              Df Sum Sq Mean Sq F value Pr(>F)    
Ekotip         1  482.9   482.9 18013.7 <2e-16 ***
Sotip          1   12.9    12.9   481.6 <2e-16 ***
Motip          1   11.5    11.5   428.0 <2e-16 ***
SdotDc         1   16.3    16.3   609.1 <2e-16 ***
Residuals   9268  248.5     0.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [124]:
owe = read_csv("OneWayExample.csv")
summary(aov(Strength ~ Sample, data=owe))

Parsed with column specification:
cols(
  Sample = col_double(),
  Strength = col_double()
)



            Df Sum Sq Mean Sq F value Pr(>F)  
Sample       1  14.31  14.305   2.895  0.097 .
Residuals   38 187.78   4.942                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [125]:
summary(pulsar_training)

     Motip             Sdotip          Ekotip            Sotip        
 Min.   :  6.188   Min.   :24.77   Min.   :-1.7380   Min.   :-1.7919  
 1st Qu.:100.984   1st Qu.:42.40   1st Qu.: 0.0245   1st Qu.:-0.1880  
 Median :115.234   Median :46.90   Median : 0.2239   Median : 0.2029  
 Mean   :111.134   Mean   :46.51   Mean   : 0.4795   Mean   : 1.7907  
 3rd Qu.:127.328   3rd Qu.:51.00   3rd Qu.: 0.4721   3rd Qu.: 0.9307  
 Max.   :189.734   Max.   :91.81   Max.   : 8.0695   Max.   :68.1016  
     MotDc              SdotDc           EkotDc           SotDc         
 Min.   :  0.2132   Min.   :  7.37   Min.   :-2.637   Min.   :  -1.977  
 1st Qu.:  1.9105   1st Qu.: 14.38   1st Qu.: 5.789   1st Qu.:  34.918  
 Median :  2.7977   Median : 18.44   Median : 8.427   Median :  83.152  
 Mean   : 12.7354   Mean   : 26.33   Mean   : 8.328   Mean   : 105.783  
 3rd Qu.:  5.4599   3rd Qu.: 28.39   3rd Qu.:10.722   3rd Qu.: 139.770  
 Max.   :211.9490   Max.   :110.64   Max.   :34.540   Max.   :119